### Import Libery for develop
this section allow load everything dependence for develop the
automate scrip and change all template AB/RE and Quotas. for this one
it's need to get files with templates before to start. for every file
it contein with 2 sheets.
- NovoSeven
- NovoEight

In [2]:
import numpy as np
import pandas as pd

# Quota


In [5]:
QU = pd.read_excel(r'C:\Users\mateo\Documents\NovoAuto\NovoAuto\Input\QUOTA.xlsx',sheet_name='')
QU.columns = QU.columns.map(lambda x: '')
customers_name = QU.iloc[:,0]

ValueError: Worksheet named '' not found

In [57]:
customers_name

0              NaN
1              NaN
2             Peso
3      Dosis (mcg)
4              NaN
          ...     
704             Cx
705            AHA
706          DFVII
707             TG
708         Others
Name: , Length: 709, dtype: object

### Extraction Customers

In [58]:
customs = []
rangos = []
for i in range(246,580,11):
    rango = slice(i+1,i+10)
    rangos.append(rango)
    customs.append(customers_name[i])
for i in range(587,616):
    customs.append(customers_name[i])
customs = [x for x in customs if customs.count(x) == 1] 
customers = {custom: rango for custom, rango in zip(customs, rangos)}

In [59]:
customers

{'DROGUERIAS CRUZ VERDE S A S': slice(247, 256, None),
 'AUDIFARMA S.A.': slice(258, 267, None),
 'ORGANIZACION VIHONCO IPS SOCIEDAD': slice(269, 278, None),
 'ORGANIZACIÓN CLINICA GENERAL DEL NORTE': slice(280, 289, None),
 'INTEGRAL SOLUTIONS SD S.A.S.': slice(291, 300, None),
 'COSMITET LTDA': slice(302, 311, None),
 'COLSUBSIDIO': slice(313, 322, None),
 'FUNDACIÓN HOSPITAL LA MISERICORDIA': slice(324, 333, None),
 'HOSPITAL PABLO TOBON URIBE': slice(335, 344, None),
 'MEDICINA INTEGRAL/ALIANZA F.': slice(346, 355, None),
 'FUNDACION SOCIAL PARA PROMOCION': slice(357, 366, None),
 'COOPERATIVA DE HOSPITALES DE': slice(368, 377, None),
 'CENTRO MEDICO IMBANACO DE CALI S.A.': slice(379, 388, None),
 'CAJA DE COMPENSACION FAMILIAR DE': slice(390, 399, None),
 'FUNDACION HOSPITAL SAN VICENTE DE PAUL ': slice(401, 410, None),
 'SALUD LLANOS IPS LTDA': slice(412, 421, None),
 'SOCIEDAD INVERSIONES FARMEDICALL': slice(423, 432, None),
 'COMFAMILIAR RISARALDA': slice(434, 443, None),
 'COB

### Extraction use_type

In [51]:
use_type= list(QU.iloc[247:256,0])

### Dates

In [53]:
date_spec = []
for a in range(2022,2028):
    for m in range(1,13):
        if m < 10:
            date = '01' + '/' + '0' + str(m) + '/' + str(a)
            date_spec.append(date)
        else:
            date = '01' + '/' +  str(m) + '/' + str(a)
            date_spec.append(date)

### Generate of tables

In [54]:
combinaciones = [(cliente, dates, ut) for cliente in customs for dates in date_spec for ut in use_type]
ab_output = pd.DataFrame(combinaciones, columns=['customer_name','date_spec','use_type'])

### Target Date

In [90]:
years = {
  '22' : slice(40,52),
  '23' : slice(53,65),
  '24' : slice(66,78),
  '25' : slice(79,91),
  '26' : slice(92,104),
  '27' : slice(105,117)  
  }  

### Add Volume

In [110]:
ab_report = []
for custom in customers:
    ab_volumenes = []
    ab_volume = []

    for year in years:
        ab_volumenes.append(QU.iloc[customers[custom],years[year]])
    ab_volumenes = pd.concat(ab_volumenes, ignore_index=True)
    ab_volumenes.columns = columna=['01','02','03','04','05','06','07','08','09','10','11','12']
    data = ab_output.loc[ab_output['customer_name'] == custom].reset_index(drop=True)
    for columna in ab_volumenes.columns:
        for valor in ab_volumenes[columna]:
            ab_volume.append(valor)
    data.loc[:, 'volume'] = pd.DataFrame(ab_volume)
    ab_report.append(data)
ab_report = pd.concat(ab_report, ignore_index=True)
ab_report['brand_name'] = 'N7'
ab_report['uom_code'] = 'MG'
ab_report['sales_rep_code'] = 'null'
ab_report['region_name'] = 'null'
ab_report = ab_report[['date_spec','sales_rep_code','region_name','customer_name', 'use_type','volume','brand_name','uom_code']]

In [111]:
ab_report

,date_spec,sales_rep_code,region_name,customer_name,use_type,volume,brand_name,uom_code
0,01/01/2022,null,null,DROGUERIAS CRUZ VERDE S A S,CHWI A PPX,130,N7,MG
1,01/01/2022,null,null,DROGUERIAS CRUZ VERDE S A S,CHWI B PPX,222,N7,MG
2,01/01/2022,null,null,DROGUERIAS CRUZ VERDE S A S,CHWI A OD,0,N7,MG
3,01/01/2022,null,null,DROGUERIAS CRUZ VERDE S A S,CHWI B OD,180,N7,MG
4,01/01/2022,null,null,DROGUERIAS CRUZ VERDE S A S,CHWI CX,NaN,N7,MG
...,...,...,...,...,...,...,...,...
20083,01/12/2027,null,null,MEDICAMENTOS ESPECIALIZADOS S.A.S.,CHWI CX,NaN,N7,MG
20084,01/12/2027,null,null,MEDICAMENTOS ESPECIALIZADOS S.A.S.,TG,NaN,N7,MG
20085,01/12/2027,null,null,MEDICAMENTOS ESPECIALIZADOS S.A.S.,HA,NaN,N7,MG
20086,01/12/2027,null,null,MEDICAMENTOS ESPECIALIZADOS S.A.S.,DFVII,NaN,N7,MG
